In [1]:
import os
import glob
import json
import quickstats
import sys

In [2]:
# make sure hh combination fw path is defined
hh_comb_fw_path = os.environ.get('hh_combination_fw_path', None)

assert hh_comb_fw_path

gen_command = True

## Step 1: Configure task

In [3]:
# specify the input timestamp to use
timestamp = "20220514"
withbr = 'with' # 'without'

paths = {
    'input'              : os.path.join(hh_comb_fw_path, "FullRun2Workspaces", "original", "HHH2022", timestamp),
    'task_options'       : os.path.join(hh_comb_fw_path, "configs", "task_options"       , "HHH2022",
                                        f"nonres_kl_kt_likelihood_{withbr}_BR_decorrelation.yaml"),
    'correlation_schemes': os.path.join(hh_comb_fw_path, "configs", "correlation_schemes", "HHH2022", 
                                        "nonres_kl_v12.json"),
    #'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_{timestamp}")
    'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_{timestamp}_{withbr}_BR_decorrelation")
}

config = {
    'resonant_type': 'nonres',
    'channels'     : ['bbbb', 'bbtautau', 'bbyy'],
    'file_expr'    : '<mass[F]>_kl',
    'blind'        : False,
    'cache'        : True,
    'experimental' : True,
    'parallel'     : -1
}

options = {
    "input_dir"    : paths['input'],
    "resonant_type": config['resonant_type'],
    "channels"     : ",".join(config['channels']),
    "outdir"       : paths['output'],
    "file_expr"    : f"\"{config['file_expr']}\"",
    "config"       : paths['task_options'],
    "parallel"     : config['parallel'],
    "skip-limit"   : ""
}

## Step 2: Prepare modified channel workspaces

In [4]:
if config["blind"]:
    options["blind"] = ""
else:
    options["unblind"] : ""
if config["experimental"]:
    options["experimental"] = ""
else:
    options["official"] : ""
if config['cache']:
    options['cache'] = ""
else:
    options['no-cache'] = ""
command_str = "HHComb process_channels " + " ".join([f"--{key} {value}" for key, value in options.items()])
print(command_str)

HHComb process_channels --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/FullRun2Workspaces/original/HHH2022/20220514 --resonant_type nonres --channels bbbb,bbtautau,bbyy --outdir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_likelihood_with_BR_decorrelation.yaml --parallel -1 --skip-limit  --experimental  --cache 


In [5]:
if not gen_command:
    os.system(command_str)

## Step 3: Prepare combined workspace

In [6]:
def return_combine_command(paths, config):
    options = {
        "input_dir"    : paths['output'],
        "resonant_type": config['resonant_type'],
        "channels"     : ",".join(config['channels']),
        "file_expr"    : f"\"{config['file_expr']}\"",
        "config"       : paths['task_options'],
        "scheme"       : paths['correlation_schemes'],
        "parallel"     : config['parallel'],
        "skip-limit"   : ""
    }
    if config["blind"]:
        options["blind"] = ""
    else:
        options["unblind"] : ""
    if config["experimental"]:
        options["experimental"] = ""
    else:
        options["official"] : ""
    if config['cache']:
        options['cache'] = ""
    else:
        options['no-cache'] = ""
    command_str = "HHComb combine_ws " + " ".join([f"--{key} {value}" for key, value in options.items()])
    print(command_str)
    return command_str

command_str = return_combine_command(paths, config)

HHComb combine_ws --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation --resonant_type nonres --channels bbbb,bbtautau,bbyy --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_likelihood_with_BR_decorrelation.yaml --scheme /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/correlation_schemes/HHH2022/nonres_kl_v12.json --parallel -1 --skip-limit  --experimental  --cache 


In [7]:
if not gen_command:
    os.system(command_str)

**Checkout the rescaled workspace paths**

In [8]:
rescaled_ws_paths = {}
for channel in config['channels']:
    ws_path = os.path.join(paths['output'], 'rescaled', config['resonant_type'], channel, "0_kl.root")
    rescaled_ws_paths[channel]  = ws_path
# rescaled workspace path for combined workspacee
ws_path = glob.glob(os.path.join(paths['output'], 'combined', config['resonant_type'], "*", "0_kl.root"))[0]
rescaled_ws_paths["combined"] = ws_path
rescaled_ws_paths

{'bbbb': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbbb/0_kl.root',
 'bbtautau': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbtautau/0_kl.root',
 'bbyy': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbyy/0_kl.root',
 'combined': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_kl.root'}

## Step 4: Prepare Asimov workspaces

**Set up the asimov workspace paths**

In [9]:
asimov_ws_paths = {}
for channel in rescaled_ws_paths:
    rescaled_ws_path = rescaled_ws_paths[channel]
    asimov_ws_path   = os.path.join(os.path.dirname(rescaled_ws_path), "0_asimov.root")
    asimov_ws_paths[channel] = asimov_ws_path
asimov_ws_paths

{'bbbb': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbbb/0_asimov.root',
 'bbtautau': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbtautau/0_asimov.root',
 'bbyy': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbyy/0_asimov.root',
 'combined': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_asimov.root'}

**Generate CLI commands**

In [10]:
options = {
    'poi'         : 'xsec_br',
    'data'        : 'combData',
    'asimov_types': '2'
}
command_str_map = {}
for channel in rescaled_ws_paths:
    input_file  = rescaled_ws_paths[channel]
    output_file = asimov_ws_paths[channel]
    channel_options = {"input_file": input_file, "output_file": output_file, **options}
    channel_command_str = "quickstats generate_standard_asimov " + \
                          " ".join([f"--{key} {value}" for key, value in channel_options.items()])
    command_str_map[channel] = channel_command_str
    print(channel_command_str, '&')
    print()

quickstats generate_standard_asimov --input_file /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbbb/0_kl.root --output_file /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbbb/0_asimov.root --poi xsec_br --data combData --asimov_types 2 &

quickstats generate_standard_asimov --input_file /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbtautau/0_kl.root --output_file /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbtautau/0_asimov.root --poi xsec_br --data combData --asimov_types 2 &

quickstats generate_standard_asimov --input_file /afs/cern.ch/work/z/zhangr/HHcomb/hh_combinat

**Generate asimov data**

In [11]:
if not gen_command:
    for channel, command_str in command_str_map.items():
        print(f"INFO: Generating asimov data for the channel \"{channel}\"")
        os.system(command_str)

## Step 5: Prepare post-fit snapshots

**Set up the fitted workspace paths**

In [12]:
fitted_ws_paths = {}
for channel in asimov_ws_paths:
    asimov_ws_path = asimov_ws_paths[channel]
    fitted_ws_path = os.path.join(os.path.dirname(asimov_ws_path), "0_fitted.root")
    fitted_ws_paths[channel] = fitted_ws_path
fitted_ws_paths

{'bbbb': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbbb/0_fitted.root',
 'bbtautau': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbtautau/0_fitted.root',
 'bbyy': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbyy/0_fitted.root',
 'combined': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_fitted.root'}

In [13]:
pois_to_scan = ["klambda", "kt", "kV", "k2V", "kF", "kH", "ktau", "klambda,kt", "kV,k2V"]
minimizer_options = {
    "retry": 2,
    "eps": 1
}
# observed and expected datasets
datasets = {
    "observed": "combData",
    "expected": "asimovData_muhat_NP_Profile"
}

In [14]:
from quickstats.components import AnalysisBase

Welcome to JupyROOT 6.24/06

RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



In [15]:
# parallel channels
import multiprocessing

def run_best_fit(channel):
    print(f"==> Channel: {channel}")
    asimov_ws_path = asimov_ws_paths[channel]
    fitted_ws_path = fitted_ws_paths[channel]
    kwargs = {
        "filename" : asimov_ws_path,
        # only need to generate snapshot for observed data
        "data_name": datasets["observed"],
        "poi_name": [],
        "config": {**minimizer_options}
    }
    analysis = AnalysisBase(**kwargs)
    # fix all pois at the beginning
    analysis.setup_parameters(fix_param="<pois>")
    analysis.save_snapshot(analysis.kTempSnapshotName)
    for poi_expr in pois_to_scan:
        print(f"==> POIs: {poi_expr}")
        analysis.setup_parameters(profile_param=poi_expr)
        analysis.nll_fit(mode=3)
        pois = poi_expr.split(",")
        snapshot_name = f"obs_bestfit_{'_'.join(pois)}"
        # save best-fit snapshot
        analysis.save_snapshot(snapshot_name)
        analysis.load_snapshot(analysis.kTempSnapshotName)
    # restore initial snapshot
    analysis.load_snapshot(analysis.kInitialSnapshotName)
    analysis.save(fitted_ws_path)

def driver_func():
    PROCESSES = len(fitted_ws_paths)
    with multiprocessing.Pool(PROCESSES) as pool:
        channels = list(fitted_ws_paths.keys())
        results = [pool.apply_async(run_best_fit, (c, )) for c in channels]

        for r in results:
            print('\t', r.get())

if not gen_command:
    driver_func()



In [16]:
driver_func()


==> Channel: bbtautau==> Channel: bbyy==> Channel: bbbb==> Channel: combined



INFO: Loaded extension module "FlexibleInterpVarMkII"
INFO: Loaded extension module "FlexibleInterpVarMkII"
INFO: Opening file "/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbtautau/0_asimov.root"
INFO: Loaded extension module "FlexibleInterpVarMkII"
INFO: Opening file "/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_asimov.root"
INFO: Loaded extension module "FlexibleInterpVarMkII"
INFO: Opening file "/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_with_BR_decorrelation/rescaled/nonres/bbbb/0_asimov.root"
INFO: Opening file "/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HH

- By now, you should have the workspace `0_fitted.root` for each channel
- Inside `0_fitted.root`, you have the observed dataset `combData` and the asimov dataset `asimovData_muhat_NP_Profile`
- For each dataset, there are the corresponding muhat snapshots designed for each likelihood scan, e.g.
  - obs_bestfit_klambda, obs_bestfit_kt, obs_bestfit_kV, obs_bestfit_k2V, obs_bestfit_klambda_kt, obs_bestfit_kV_k2V
  - exp_bestfit_klambda, exp_bestfit_kt, exp_bestfit_kV, exp_bestfit_k2V, exp_bestfit_klambda_kt, exp_bestfit_kV_k2V

## Step 6: Run likelihood scans

**Only commands are shown here since the jobs are computationally intensive**

In [193]:
from quickstats.parsers import ParamParser

# use '_job' + suffix to split scan range
param_expr_maps = {
    'klambda': "^klambda=-6_12_0.2^",
    "klambda_kt_profile": "^klambda=-16_20_0.4,kt^",
    "klambda_kt_2D_job1": "^klambda=-15_-5_0.2,kt=0.6_1.6_0.1^",
    "klambda_kt_2D_job2": "^klambda=-5_5_0.2,kt=0.6_1.6_0.1^",
    "klambda_kt_2D_job3": "^klambda=5_15_0.2,kt=0.6_1.6_0.1^",
    "klambda_kt_2D_job4": "^klambda=15_20_0.2,kt=0.6_1.6_0.1^",
    "kt": "^kt=-2_6_0.1^",
    "kF": "^kF=-8_15_0.1^",
    "kV": "^kV=-2.5_2.5_0.1^",
    "k2V": "^k2V=-3_5_0.1^",
    "kV_k2V_2D_job1": "^kV=-3_3_0.2,k2V=-4_-2_0.2^",
    "kV_k2V_2D_job2": "^kV=-3_3_0.2,k2V=-2_0_0.2^",
    "kV_k2V_2D_job3": "^kV=-3_3_0.2,k2V=0_2_0.2^",
    "kV_k2V_2D_job4": "^kV=-3_3_0.2,k2V=2_4_0.2^",
    "kV_k2V_2D_job5": "^kV=-3_3_0.2,k2V=4_6_0.2^",
    "kV_k2V_2D_job6": "^kV=-3_3_0.2,k2V=6_8_0.2^",
    "kV_k2V_2D_job7": "^kV=-3_3_0.2,k2V=8_10_0.2^",
    "kV_k2V_2D_final": "^kV=-3_3_0.2,k2V=-4_10_0.2^",
}

likelihood_scan_cmds = {}
for channel in fitted_ws_paths:
    likelihood_scan_cmds[channel] = {}
    fitted_ws_path = fitted_ws_paths[channel]
    for dataset_type, dataset_name in datasets.items():
        likelihood_scan_cmds[channel][dataset_type] = {}
        for key, expr in param_expr_maps.items():
            if condor and 'final' in key:
                continue
            if not condor and 'job' in key:
                continue
            if 'final' in key:
                outdir = os.path.join(paths['output'], 'likelihood', dataset_type, channel, key.replace('_final', ''))
            else:
                outdir = os.path.join(paths['output'], 'likelihood', dataset_type, channel, (key+'1')[:key.find('_job')])
            pois = ParamParser._get_param_str_attributes(expr)
            pois = [p.replace('^', '') for p in pois]
            if dataset_type == "observed":
                snapshot_name = f"obs_bestfit_{'_'.join(pois)}"
            else:
                snapshot_name = dataset_name
            options = {
                "input_file" : fitted_ws_path,
                "data": dataset_name,
                "param_expr": expr,
                'outdir': outdir,
                'snapshot': snapshot_name,
                **minimizer_options
            }
            cmd_str = "quickstats likelihood_scan " + \
                      " ".join([f"--{key} {value}" for key, value in options.items()])
            likelihood_scan_cmds[channel][dataset_type][key] = cmd_str
            
            
condor = True
if condor:
    original_stdout = sys.stdout
    sys.stdout = open('job.txt', 'w')
for channel in likelihood_scan_cmds:
    if not condor:
        print("#########################################################################################")
    print(f"# Channel: {channel}")
    for dataset_type in likelihood_scan_cmds[channel]:
        print(f"# Dataset: {dataset_type}")
        for key in likelihood_scan_cmds[channel][dataset_type]:
            if not condor:
                print(f"# Scan parameters: {key}")
            cmd_str = likelihood_scan_cmds[channel][dataset_type][key]
            if not condor:
                print(cmd_str.replace('^', '"'))
            else:
                print("Arguments =", cmd_str.replace(' ', '____'))
                print("Queue 1")
        print()
    if not condor:
        print("#########################################################################################")

if condor: 
    sys.stdout = original_stdout

## Step 7: Run cross section scans

In [194]:
# specify the input timestamp to use
timestamp = "20220514_noSgHparam"
withbr = 'without' # 'without'

paths = {
    'input'              : os.path.join(hh_comb_fw_path, "FullRun2Workspaces", "original", "HHH2022", timestamp),
    'task_options'       : os.path.join(hh_comb_fw_path, "configs", "task_options"       , "HHH2022",
                                        f"nonres_kl_kt_xsection.yaml"),
    'correlation_schemes': os.path.join(hh_comb_fw_path, "configs", "correlation_schemes", "HHH2022", 
                                        "nonres_kl_v12.json"),
    'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_{timestamp}")
}

config = {
    'resonant_type': 'nonres',
    'channels'     : ['bbbb', 'bbtautau', 'bbyy'],
    'file_expr'    : '<mass[F]>_kl',
    'blind'        : False,
    'cache'        : True,
    'experimental' : True,
    'parallel'     : -1,
    'type'         : 'xsec'
}
options = {
    "input_dir"    : paths['input'],
    "resonant_type": config['resonant_type'],
    "channels"     : ",".join(config['channels']),
    "outdir"       : paths['output'],
    "file_expr"    : f"\"{config['file_expr']}\"",
    "config"       : paths['task_options'],
    "parallel"     : config['parallel'],
    "skip-limit"   : "",
}

In [195]:
if config["blind"]:
    options["blind"] = ""
else:
    options["unblind"] : ""
if config["experimental"]:
    options["experimental"] = ""
else:
    options["official"] : ""
if config['cache']:
    options['cache'] = ""
else:
    options['no-cache'] = ""
command_str1 = "HHComb process_channels " + " ".join([f"--{key} {value}" for key, value in options.items()])
print(command_str1)
# os.system(command_str1)

if config['type'] == 'xsec':
    options['fix'] = f"\"THEO_XS_fixmu_*=0,alpha_THEO_XS_PDFalphas_VBFSMHH*=0,alpha_THEO_XS_PDFalphas_ggFSMHH*=0,alpha_THEO_XS_SCALEMTop_ggFSMHH*=0,THEO_XS_COMBINED_HH_ggF*=0,THEO_XS_PDFalphas_HH_VBF*=0,THEO_XS_PDFalphas_HH_ggF*=0,THEO_XS_SCALE_HH_VBF*=0\""


HHComb process_channels --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/FullRun2Workspaces/original/HHH2022/20220514_noSgHparam --resonant_type nonres --channels bbbb,bbtautau,bbyy --outdir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_noSgHparam --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_xsection.yaml --parallel -1 --skip-limit  --experimental  --cache 


In [196]:
command_str = return_combine_command(paths, config)

HHComb combine_ws --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_noSgHparam --resonant_type nonres --channels bbbb,bbtautau,bbyy --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_xsection.yaml --scheme /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/correlation_schemes/HHH2022/nonres_kl_v12.json --parallel -1 --skip-limit  --experimental  --cache 


In [197]:
rescaled_ws_paths = {}
for channel in config['channels']:
    ws_path = os.path.join(paths['output'], 'rescaled', config['resonant_type'], channel, "0_kl.root")
    rescaled_ws_paths[channel] = ws_path
# rescaled workspace path for combined workspacee
ws_path = glob.glob(os.path.join(paths['output'], 'combined', config['resonant_type'], "*", "0_kl.root"))[0]
rescaled_ws_paths["combined"] = ws_path
rescaled_ws_paths

{'bbbb': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_noSgHparam/rescaled/nonres/bbbb/0_kl.root',
 'bbtautau': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_noSgHparam/rescaled/nonres/bbtautau/0_kl.root',
 'bbyy': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_noSgHparam/rescaled/nonres/bbyy/0_kl.root',
 'combined': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220514_noSgHparam/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_kl.root'}

In [198]:
# use '_job' + suffix to split scan range
param_expr_maps = {
    'klambda_job1': "^klambda=-15_0_0.2^",
    'klambda_job2': "^klambda=0_20_0.2^",
    'klambda_final': "^klambda=-15_20_0.2^",
    'k2v_job2': "^k2V=-2_4_0.2^",
    'klambdak2v_job1': "^klambda=-15_-13_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job2': "^klambda=-13_-11_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job3': "^klambda=-11_-9_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job4': "^klambda=-9_-7_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job5': "^klambda=-7_-5_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job6': "^klambda=-5_-3_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job7': "^klambda=-3_-1_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job8': "^klambda=-1_1_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job9': "^klambda=1_3_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job10': "^klambda=3_5_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job11': "^klambda=5_7_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job12': "^klambda=7_9_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job13': "^klambda=9_11_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job14': "^klambda=11_13_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job15': "^klambda=13_15_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job16': "^klambda=15_17_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job17': "^klambda=17_19_0.2,k2V=-2_4_0.2^",
    'klambdak2v_job18': "^klambda=19_20_0.2,k2V=-2_4_0.2^",
    'klambdak2v_final': "^klambda=-15_20_0.2,k2V=-2_4_0.2^",
    'k2vkv_job1': "^kV=-3_-2_0.2,k2V=-2_6_0.2^",
    'k2vkv_job2': "^kV=-2_-1_0.2,k2V=-2_6_0.2^",
    'k2vkv_job3': "^kV=-1_-0_0.2,k2V=-2_6_0.2^",
    'k2vkv_job4': "^kV=-0_1_0.2,k2V=-2_6_0.2^",
    'k2vkv_job5': "^kV=1_2_0.2,k2V=-2_6_0.2^",
    'k2vkv_job6': "^kV=2_3_0.2,k2V=-2_6_0.2^",
    'k2vkv_final': "^kV=-3_3_0.2,k2V=-2_6_0.2^",
}

options = {
    'poi'         : 'xsec_br',
    'data'        : 'combData',
    'unblind'     : '',
    'fix'         : "^THEO_XS_fixmu_*=0,alpha_THEO_XS_PDFalphas_VBFSMHH*=0,alpha_THEO_XS_PDFalphas_ggFSMHH*=0,alpha_THEO_XS_SCALEMTop_ggFSMHH*=0,THEO_XS_COMBINED_HH_ggF*=0,THEO_XS_PDFalphas_HH_VBF*=0,THEO_XS_PDFalphas_HH_ggF*=0,THEO_XS_SCALE_HH_VBF*=0^",
}
command_str_map = {}

condor = True
if condor:
    original_stdout = sys.stdout
    sys.stdout = open('job2.txt', 'w')

for channel in rescaled_ws_paths:
    print(f"# Channel: {channel}")
    input_path  = rescaled_ws_paths[channel]
    for key, expr in param_expr_maps.items():
        
        if condor and 'final' in key:
            continue
        if not condor and 'job' in key:
            continue
        if 'final' in key:
            outdir = os.path.join(paths['output'], 'xsection_scan', channel, key.replace('_final', ''))
        else:
            outdir = os.path.join(paths['output'], 'xsection_scan', channel, (key+'1')[:key.find('_job')])

        channel_options = {"input_path": input_path, "outdir": outdir, "param_expr": expr, **options}
        channel_command_str = "quickstats limit_scan " + \
                              " ".join([f"--{key} {value}" for key, value in channel_options.items()])
        command_str_map[channel] = channel_command_str


        if not condor:
            print(channel_command_str)
        else:
            print("Arguments =", channel_command_str.replace(' ', '____'))
            print("Queue 1")
    print()

if condor: 
    sys.stdout = original_stdout